# Polygon Area Calculation

Calculate the selected polygon area by calculating back ground (bg) pixels.  
API is always providing a bounding box (BB) which polygon is inside.  
To estimate the actual polygon area, the following approach going to be taken.  
__Approach 1:__    
1. Calculate the bg pixel count which is stored as 0.0
2. Get the BB pixel count -> (rows*cols)
3. Polygon area = BB pixel - bg pixel

In [1]:
import numpy as np
import numpy.ma as ma
import rioxarray as rxr
import earthpy as et
import earthpy.spatial as es

## Gampola Land

In [133]:
img_path = r'C:\Users\User\OneDrive - Engenuity (Pvt) Ltd\Github\SPE-testing\Cloud cover\Data\4214373c5795de2e9483de852fe0920e'
file = 'response.tiff'
file = rxr.open_rasterio(img_path+'/'+file)

In [134]:
print(file.shape)

(5, 46, 80)


In [135]:
file.values

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

    

In [136]:
_, rows, cols = file.shape
print(rows)
print(cols)

46
80


In [137]:
bb = file[3].values.copy()
bb.shape

(46, 80)

In [138]:
for i in range(0,rows-1):
    for j in range(0,cols-1):
        if bb[i][j] > 0.0:
            bb[i][j] = 0
        else:
            bb[i][j] = 1

In [139]:
bb_list = bb.tolist()
bg_pixel = sum(x.count(1) for x in bb_list)
bg_pixel

1020

In [140]:
land_pixel = (rows*cols)-bg_pixel
land_pixel

2660

In [141]:
res = 10
land_area = (land_pixel*res*res)/1000000 # in km2
print(land_area, "sqr kilometers")

0.266 sqr kilometers


## Sri Lanka

In [123]:
img_path = r'C:\Users\User\OneDrive - Engenuity (Pvt) Ltd\Github\SPE-testing\Sentinel-hub API\Data\c63cc1c7352bd86d218dbe557a56aa21'
file = 'response.tiff'
file = rxr.open_rasterio(img_path+'/'+file)
print(file.shape)
file.values

(4, 2476, 1453)


array([[[1196., 1197., 1198., ...,    0.,    0.,    0.],
        [1199., 1196., 1197., ...,    0.,    0.,    0.],
        [1204., 1205., 1213., ...,    0.,    0.,    0.],
        ...,
        [   0.,    0.,    0., ...,    0.,    0.,    0.],
        [   0.,    0.,    0., ...,    0.,    0.,    0.],
        [   0.,    0.,    0., ...,    0.,    0.,    0.]],

       [[ 923.,  925.,  925., ...,    0.,    0.,    0.],
        [ 928.,  925.,  926., ...,    0.,    0.,    0.],
        [ 936.,  938.,  941., ...,    0.,    0.,    0.],
        ...,
        [   0.,    0.,    0., ...,    0.,    0.,    0.],
        [   0.,    0.,    0., ...,    0.,    0.,    0.],
        [   0.,    0.,    0., ...,    0.,    0.,    0.]],

       [[ 403.,  404.,  404., ...,    0.,    0.,    0.],
        [ 404.,  405.,  404., ...,    0.,    0.,    0.],
        [ 410.,  412.,  412., ...,    0.,    0.,    0.],
        ...,
        [   0.,    0.,    0., ...,    0.,    0.,    0.],
        [   0.,    0.,    0., ...,    0.,    

In [93]:
bb = file[3].values.copy()
print(bb.shape)
rows, cols = bb.shape
print(rows)
print(cols)

(2476, 1453)
2476
1453


In [94]:
for i in range(0,rows-1):
    for j in range(0,cols-1):
        if bb[i][j] > 0.0:
            bb[i][j] = 0
        else:
            bb[i][j] = 1

In [95]:
bb_list = bb.tolist()
bg_pixel = sum(x.count(1) for x in bb_list)
bg_pixel

333614

In [96]:
land_pixel = (rows*cols)-bg_pixel
land_pixel

3264014

In [98]:
res = 135
land_area = np.round((land_pixel*res*res)/1000000,2) # in km2
print(land_area, "sqr kilometers")

59486.66 sqr kilometers


## Polygon Area Function

In [151]:
def poly_area(req,resol):
    bb = req[0].values
    r, c = bb.shape
    
    for i in range(0,r-1):
        for j in range(0,c-1):
            if bb[i][j] > 0.0:
                bb[i][j] = 0
            else:
                bb[i][j] = 1
            
    bb_list = bb.tolist()
    bg_pixel = sum(x.count(1) for x in bb_list)
    poly_pixel = (r*c)-bg_pixel
    poly_area = np.round((poly_pixel*res*res)/1000000,2) # in km2
    print("Polygon Area",poly_area, "km2")
    
    return poly_area

Test with Gampola site

In [152]:
poly_area(file,10)

Polygon Area 0.11 km2


0.11